In [1]:
import os

In [15]:
# normal imgs
fedora_path = "/home/sisa/sat-data/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"
extern_path = "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/MOD_RGB_corrected_15percent/"

# predictions
extern_prediction_path = "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_DEC2023_Predictions_Binary/"


# experiment 2 path
extern_MODIS_path =  "/mnt/f/Bachelorarbeit/sat-data/modis/MODIS_MOD09GA_GERITAFRA_31day_DEC2023/"

path = extern_MODIS_path

In [16]:
files = os.listdir(path)#
files

['MOD09GA.A2023360.h18v04.061.2023362134719.hdf',
 'MOD09GA.A2023363.h18v04.061.2023365041515.hdf',
 'MOD09GA.A2023354.h18v04.061.2023356042206.hdf',
 'MOD09GA.A2023361.h18v04.061.2023363043955.hdf',
 'MOD09GA.A2023337.h18v04.061.2023339023523.hdf',
 'MOD09GA.A2023353.h18v04.061.2023355063849.hdf',
 'MOD09GA.A2023356.h18v04.061.2023361000333.hdf',
 '8377414216-download.sh',
 'MOD09GA.A2023362.h18v04.061.2023364042452.hdf',
 'MOD09GA.A2023341.h18v04.061.2023343024753.hdf',
 'MOD09GA.A2023342.h18v04.061.2023344024915.hdf',
 'MOD09GA.A2023352.h18v04.061.2023354042819.hdf',
 'MOD09GA.A2023358.h18v04.061.2023360025923.hdf',
 'MOD09GA.A2023350.h18v04.061.2023352030330.hdf',
 'MOD09GA.A2023364.h18v04.061.2024001025343.hdf',
 'MOD09GA.A2023346.h18v04.061.2023348030312.hdf',
 'MOD09GA.A2023347.h18v04.061.2023349025736.hdf',
 'MOD09GA.A2023365.h18v04.061.2024004181336.hdf',
 'MOD09GA.A2023336.h18v04.061.2023338023416.hdf',
 'MOD09GA.A2023348.h18v04.061.2023350024202.hdf',
 'MOD09GA.A2023345.h18v

In [37]:
hdf_files = sorted([f for f in files if f[-4:] == ".hdf"])


## Definitionen

In [83]:
def generate_ql_modis_commands(folder="../MODIS_DEC2023_Predictions_Binary", filename="", suffix="", extension=".tiff", date=1):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    suffix like     _merged_2400_binary_M1_x_prediction_bin_mask
    extensions like .tiff
    date start counting at 1!
    """
    ql_modis_template = f"--ql-modis=(--file {folder}{filename}{suffix}{extension} --date {date})"
    return ql_modis_template


In [91]:
def generate_ql_img_commands(folder="../MODIS_DEC2023_Predictions_Binary/", filename="", suffix="", extension=".tiff", date=1):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    suffix like     _merged_2400_binary_M1_x_prediction_bin_mask
    extensions like .tiff
    date start counting at 1!
    """
    ql_img_template = f"--ql-img=(--file {folder}{filename}{suffix}{extension} --date {date} --interp-ranges=[1,1] --non-interp-ranges=[0,0])"
    return ql_img_template

In [100]:
def generate_modis_hdf_commands(folder="../", filename="", layers=(11,12,13,14,15,16,17), date=1, extension=".hdf", suffix=""):
    """
    filename like   MOD09GA.A2023335.h18v04.061.2023340033530
    extensions like .hdf
    date start counting at 1!
    """
    layer_string = "("
    for layer in layers:
        layer_string += str(layer) + " "
    layer_string += ")"

    modis_hdf_template = f"--img=(--file {folder}{filename}{suffix}{extension} -l {layer_string} --date {date})"
    return modis_hdf_template

## Generierung

In [101]:
# definieren der tage
start = 6
stop  = 19

In [102]:
modis_hdf_commands=[]
ql_modis_commands=[]
original_modis_ql_commands=[]
ql_img_commands = []
modis_tiff_commands=[]

model_suffix= "_merged_2400_binary_M1_240605215803_prediction_bin_mask" #  "_merged_2400_Unet-v2-binary_240526173722_prediction_bin_mask"
original_ql_suffix = "_state_1km_1_2400"
tiff_suffix="_merged_2400"


for i, file in enumerate(hdf_files[start:stop]):
    # remember, i has to be ++ed for imfu
    #print(i+1, file)
    filename = file[:-4]
    extension = ".hdf"
    date = i+1

    modis_hdf_command=generate_modis_hdf_commands(filename=filename, date=date, extension=extension)
    modis_hdf_commands.append(modis_hdf_command)

    modis_tiff_command=generate_modis_hdf_commands(filename=filename, extension=".tiff", layers=(0,1,2,3,4,5,6), date=date, suffix=tiff_suffix)
    modis_tiff_commands.append(modis_tiff_command)
    # quality layers
    ql_modis_command = generate_ql_modis_commands(filename=filename, suffix=model_suffix, date=date)
    ql_modis_commands.append(ql_modis_command)

    original_ql_modis_command = generate_ql_modis_commands(folder="../",filename=filename, suffix=original_ql_suffix, date=date)
    original_modis_ql_commands.append(original_ql_modis_command)

    ql_img_command = generate_ql_img_commands(filename=filename, suffix=model_suffix, date=date)
    ql_img_commands.append(ql_img_command)

In [103]:
# print commands you want listed
for c in modis_hdf_commands:
    print(c)

--img=(--file ../MOD09GA.A2023341.h18v04.061.2023343024753.hdf -l (11 12 13 14 15 16 17 ) --date 1)
--img=(--file ../MOD09GA.A2023342.h18v04.061.2023344024915.hdf -l (11 12 13 14 15 16 17 ) --date 2)
--img=(--file ../MOD09GA.A2023343.h18v04.061.2023345024918.hdf -l (11 12 13 14 15 16 17 ) --date 3)
--img=(--file ../MOD09GA.A2023344.h18v04.061.2023346030519.hdf -l (11 12 13 14 15 16 17 ) --date 4)
--img=(--file ../MOD09GA.A2023345.h18v04.061.2023347025532.hdf -l (11 12 13 14 15 16 17 ) --date 5)
--img=(--file ../MOD09GA.A2023346.h18v04.061.2023348030312.hdf -l (11 12 13 14 15 16 17 ) --date 6)
--img=(--file ../MOD09GA.A2023347.h18v04.061.2023349025736.hdf -l (11 12 13 14 15 16 17 ) --date 7)
--img=(--file ../MOD09GA.A2023348.h18v04.061.2023350024202.hdf -l (11 12 13 14 15 16 17 ) --date 8)
--img=(--file ../MOD09GA.A2023349.h18v04.061.2023351023935.hdf -l (11 12 13 14 15 16 17 ) --date 9)
--img=(--file ../MOD09GA.A2023350.h18v04.061.2023352030330.hdf -l (11 12 13 14 15 16 17 ) --date 10)